In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

filepath = "path.xlsx"
df = pd.read_excel(filepath, index_col=0)
df.tail()

,Grupo,Sexo,Idade,Massa,Estatura,IMC,Gait_Speed_TS_T0,Gait_Speed_TS_T3,Gait_Speed_TS_T6,Gait_Speed_TS_T9,...,StroopColor_ortost_T0,StroopColor_ortost_T3,StroopColor_ortost_T6,StroopColor_ortost_T9,StroopColor_ortost_T12,StroopColorEffectOrt_T0,StroopColorEffectOrt_T3,StroopColorEffectOrt_T6,StroopColorEffectOrt_T9,StroopColorEffectOrt_T12
ID,,,,,,,,,,,,,,,,,,,,,
s56,GE,F,70,79.0,1.54,33.310845,1.13,NaN,NaN,NaN,...,87.8,NaN,NaN,NaN,NaN,-54.305800,NaN,NaN,NaN,NaN
s57,GC,M,78,69.4,1.69,24.298869,1.59,NaN,NaN,NaN,...,202.6,NaN,NaN,NaN,NaN,-342.358079,NaN,NaN,NaN,NaN
s58,GC,F,71,75.0,1.59,29.666548,1.27,1.15,,1.15,...,53.6,53.3,42.3,49.4,NaN,-109.375000,-125.847458,-81.545064,-94.488189,NaN
s59,GC,F,73,55.5,1.58,22.232014,1.25,NaN,1.47,1.32,...,103.5,NaN,149.8,142.1,95.0,-175.265957,NaN,-255.819477,-299.157303,-157.452575
s60,GC,F,65,59.4,1.46,27.866391,1.26,NaN,1.53,1.36,...,64.5,NaN,59.4,52.1,50.0,-113.576159,NaN,-102.730375,-84.098940,-76.056338


In [2]:
list(df['Grupo'].unique())

['GC', 'GE']

In [3]:
df.loc[df['Grupo'] == 'GC', 'Label'] = int(1)
df.loc[df['Grupo'] == 'GE', 'Label'] = int(0)
df['Label'] = df['Label'].astype('int64')

In [4]:
df['Label'].head()

ID
s01    1
s02    1
s03    0
s04    1
s05    0
Name: Label, dtype: int64

In [99]:
from sklearn.model_selection import cross_val_score
from skrebate import ReliefF
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


def generate_experiment(df, classifier, relief=True):
    features, labels = df.drop(['Label'], axis=1).astype('float64').values, df['Label'].astype('int64').values
    
    if relief:
        clf = make_pipeline(
            *[ReliefF(n_features_to_select=10, n_neighbors=100, verbose=False, n_jobs=8), classifier],
            verbose=False,
        )
        clf.fit(features, labels)
    else:
        clf = make_pipeline(
            *[classifier],
            verbose=False,
        )
        clf.fit(features, labels)

    scores = cross_val_score(clf, features, labels)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    scores = cross_val_score(clf, features, labels, scoring='f1_micro')
    print("F1 Micro: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    scores = cross_val_score(clf, features, labels, scoring='precision')
    print("Precision: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    scores = cross_val_score(clf, features, labels, scoring='recall')
    print("Recall: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    scores = cross_val_score(clf, features, labels, scoring='roc_auc')
    print("ROC: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

    if relief:
        relief = clf.get_params()['steps'][0][1]
        dfcolumns = pd.DataFrame(df.drop(['Label'], axis=1).columns)
        dfscores = pd.DataFrame(relief.feature_importances_)

        featureScores = pd.concat([dfcolumns, dfscores], axis=1)
        featureScores.columns = ['Features','Score']
        df_features_scores = featureScores.sort_values("Score", ascending=False)
        return df_features_scores[:10]

# Experiments
- The goal is to find the features that most represent the final evaluation state
- Use T0, T3, T6, T9, T12 to predict the Group (GC or GE)
    
# Preprocessing
- Remove patients with NaNs on T12 evaluations
- Replace NaNs by an average of the previous and posterior evaluation moment
- **17 subjects** by considering only subjects that have all the measures.
- **31 subjects** by considering subjects that may have missed evaluations but have the T12.
- **42 subjects** by considering subjects that may have missed evaluations and predicting the T9 state instead of T12

# Experiment #1
- Using only measurements T0 and T2 to predict the group (GC or GE)

In [100]:
drop_cols = [col for col in df.columns if "T3" in col or "T6" in col or "T9" in col]
T0_T12_df = df.drop(labels=drop_cols, axis=1)
T0_T12_df = T0_T12_df.drop(labels=['Grupo', 'Sexo'], axis=1)
T0_T12_df = T0_T12_df.dropna()

In [101]:
print(len(T0_T12_df))
experiment_1_columns = T0_T12_df.columns
T0_T12_df.head()

22


,Idade,Massa,Estatura,IMC,Gait_Speed_TS_T0,Gait_Speed_TS_T12,Gait_Variability_TS_T0,Gait_Variability_TS_T12,Gait_Assimetry_TS_T0,Gait_Assimetry_TS_T12,...,StroopColor_T12,StroopColorEffect_T0,StroopColorEffect_T12,StroopPB_ortost_T0,StroopPB_ortost_T12,StroopColor_ortost_T0,StroopColor_ortost_T12,StroopColorEffectOrt_T0,StroopColorEffectOrt_T12,Label
ID,,,,,,,,,,,,,,,,,,,,,
s02,60,63.0,1.64,23.423557,1.41,1.31,2.57,2.22,-1.9,5.5,...,41.3,-50.488599,-69.958848,34.0,22.9,56.6,44.3,-66.470588,-93.449782,1
s03,70,62.9,1.59,24.880345,1.40,1.44,1.90,2.18,1.7,1.1,...,63.4,-140.909091,-129.710145,33.5,29.0,97.1,60.9,-189.850746,-110.000000,0
s04,64,81.0,1.58,32.446723,1.34,1.46,2.43,3.00,3.1,-1.7,...,50.4,-110.511364,-93.846154,31.2,27.0,71.1,58.8,-127.884615,-117.777778,1
s06,68,74.0,1.54,31.202564,1.40,1.39,3.66,2.26,0.6,-1.3,...,83.7,-418.000000,-142.608696,56.7,34.4,130.0,78.3,-245.149912,-127.616279,1
s09,69,56.0,1.50,24.888889,1.45,1.38,1.42,1.57,2.4,-2.5,...,66.2,-157.512953,-89.142857,38.6,32.5,78.6,72.1,-103.626943,-121.846154,0


In [103]:
generate_experiment(T0_T12_df, classifier=RandomForestClassifier(n_estimators=100))

Accuracy: 0.48 (+/- 0.62)
F1 Micro: 0.49 (+/- 0.48)
Precision: 0.53 (+/- 0.80)
Recall: 0.67 (+/- 0.56)
ROC: 0.62 (+/- 0.53)


,Features,Score
108,BIOF_T0,0.297521
112,CTSIB_Total_T0,0.264463
135,SF36_Pain_T12,0.206612
9,Gait_Assimetry_TS_T12,0.186931
123,GDS_T12,0.173554
61,Gait_Assimetry_DTPF_T12,0.151534
114,STS_5X_T0,0.143054
149,StroopColor_T12,0.132894
35,Gait_Assimetry_DTPV_T12,0.126865
92,EERE_T0,0.123967


In [104]:
generate_experiment(T0_T12_df, classifier=SVC(kernel='linear'))

Accuracy: 0.64 (+/- 0.55)
F1 Micro: 0.64 (+/- 0.55)
Precision: 0.70 (+/- 0.53)
Recall: 0.67 (+/- 0.56)
ROC: 0.65 (+/- 0.60)


,Features,Score
108,BIOF_T0,0.297521
112,CTSIB_Total_T0,0.264463
135,SF36_Pain_T12,0.206612
9,Gait_Assimetry_TS_T12,0.186931
123,GDS_T12,0.173554
61,Gait_Assimetry_DTPF_T12,0.151534
114,STS_5X_T0,0.143054
149,StroopColor_T12,0.132894
35,Gait_Assimetry_DTPV_T12,0.126865
92,EERE_T0,0.123967


In [105]:
generate_experiment(T0_T12_df, classifier=SVC(kernel='sigmoid'))

Accuracy: 0.61 (+/- 0.47)
F1 Micro: 0.61 (+/- 0.47)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.51 (+/- 0.66)
Recall: 0.80 (+/- 0.80)
ROC: 0.70 (+/- 0.80)


,Features,Score
108,BIOF_T0,0.297521
112,CTSIB_Total_T0,0.264463
135,SF36_Pain_T12,0.206612
9,Gait_Assimetry_TS_T12,0.186931
123,GDS_T12,0.173554
61,Gait_Assimetry_DTPF_T12,0.151534
114,STS_5X_T0,0.143054
149,StroopColor_T12,0.132894
35,Gait_Assimetry_DTPV_T12,0.126865
92,EERE_T0,0.123967


In [106]:
generate_experiment(T0_T12_df, classifier=SVC(kernel='poly'))

Accuracy: 0.46 (+/- 0.61)
F1 Micro: 0.46 (+/- 0.61)
Precision: 0.52 (+/- 0.82)
Recall: 0.47 (+/- 0.65)
ROC: 0.52 (+/- 0.73)


,Features,Score
108,BIOF_T0,0.297521
112,CTSIB_Total_T0,0.264463
135,SF36_Pain_T12,0.206612
9,Gait_Assimetry_TS_T12,0.186931
123,GDS_T12,0.173554
61,Gait_Assimetry_DTPF_T12,0.151534
114,STS_5X_T0,0.143054
149,StroopColor_T12,0.132894
35,Gait_Assimetry_DTPV_T12,0.126865
92,EERE_T0,0.123967


## Without Relief (feature selection)

In [108]:
generate_experiment(T0_T12_df, classifier=RandomForestClassifier(n_estimators=100), relief=False)

Accuracy: 0.59 (+/- 0.31)
F1 Micro: 0.53 (+/- 0.48)
Precision: 0.60 (+/- 0.78)
Recall: 0.57 (+/- 0.78)
ROC: 0.55 (+/- 0.90)


In [109]:
generate_experiment(T0_T12_df, classifier=SVC(kernel='linear'), relief=False)

Accuracy: 0.50 (+/- 0.34)
F1 Micro: 0.50 (+/- 0.34)
Precision: 0.50 (+/- 0.21)
Recall: 0.57 (+/- 0.45)
ROC: 0.55 (+/- 0.53)


In [110]:
generate_experiment(T0_T12_df, classifier=SVC(kernel='sigmoid'), relief=False)

Accuracy: 0.41 (+/- 0.18)
F1 Micro: 0.41 (+/- 0.18)
Precision: 0.18 (+/- 0.45)
Recall: 0.30 (+/- 0.80)
ROC: 0.28 (+/- 0.08)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [111]:
generate_experiment(T0_T12_df, classifier=SVC(kernel='poly'), relief=False)

Accuracy: 0.36 (+/- 0.37)
F1 Micro: 0.36 (+/- 0.37)
Precision: 0.08 (+/- 0.32)
Recall: 0.20 (+/- 0.80)
ROC: 0.37 (+/- 0.39)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Experiment 2
- Using only measurements from T0 to predict the group (GC or GE)

In [112]:
drop_cols = [col for col in df.columns if "T3" in col or "T6" in col or "T9" in col or "T12" in col]
T0_df = df.drop(labels=drop_cols, axis=1)
T0_df = T0_df.drop(labels=['Grupo', 'Sexo', 'Missing Values'], axis=1)
T0_df = T0_df.dropna()

In [114]:
generate_experiment(T0_df, classifier=SVC(kernel='poly'))

Accuracy: 0.39 (+/- 0.15)
F1 Micro: 0.39 (+/- 0.15)
Precision: 0.30 (+/- 0.36)
Recall: 0.39 (+/- 0.68)
ROC: 0.23 (+/- 0.33)


,Features,Score
56,BIOF_T0,0.099112
58,CTSIB_Total_T0,0.081361
15,SwingSpeed_D_TS_T0,0.054231
64,SF36_PF_T0,0.042792
19,Gait_Assimetry_DTPV_T0,0.039463
52,BEOA_T0,0.036982
28,SwingSpeed_D_DTPV_T0,0.035598
71,SF36_GHP_T0,0.032914
14,PeakAngVelocity_D_TS_T0,0.031757
29,MinToe_D_DTPV_T0,0.031065


In [115]:
generate_experiment(T0_df, classifier=SVC(kernel='sigmoid'))

Accuracy: 0.48 (+/- 0.13)
F1 Micro: 0.48 (+/- 0.13)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.29 (+/- 0.49)
Recall: 0.40 (+/- 0.76)
ROC: 0.40 (+/- 0.34)


,Features,Score
56,BIOF_T0,0.099112
58,CTSIB_Total_T0,0.081361
15,SwingSpeed_D_TS_T0,0.054231
64,SF36_PF_T0,0.042792
19,Gait_Assimetry_DTPV_T0,0.039463
52,BEOA_T0,0.036982
28,SwingSpeed_D_DTPV_T0,0.035598
71,SF36_GHP_T0,0.032914
14,PeakAngVelocity_D_TS_T0,0.031757
29,MinToe_D_DTPV_T0,0.031065


In [116]:
generate_experiment(T0_df, classifier=SVC(kernel='linear'))

Accuracy: 0.52 (+/- 0.27)
F1 Micro: 0.52 (+/- 0.27)
Precision: 0.45 (+/- 0.46)
Recall: 0.51 (+/- 0.61)
ROC: 0.58 (+/- 0.38)


,Features,Score
56,BIOF_T0,0.099112
58,CTSIB_Total_T0,0.081361
15,SwingSpeed_D_TS_T0,0.054231
64,SF36_PF_T0,0.042792
19,Gait_Assimetry_DTPV_T0,0.039463
52,BEOA_T0,0.036982
28,SwingSpeed_D_DTPV_T0,0.035598
71,SF36_GHP_T0,0.032914
14,PeakAngVelocity_D_TS_T0,0.031757
29,MinToe_D_DTPV_T0,0.031065


In [118]:
generate_experiment(T0_df, classifier=RandomForestClassifier(n_estimators=100))

Accuracy: 0.51 (+/- 0.26)
F1 Micro: 0.46 (+/- 0.11)
Precision: 0.56 (+/- 0.49)
Recall: 0.58 (+/- 0.27)
ROC: 0.45 (+/- 0.24)


,Features,Score
56,BIOF_T0,0.099112
58,CTSIB_Total_T0,0.081361
15,SwingSpeed_D_TS_T0,0.054231
64,SF36_PF_T0,0.042792
19,Gait_Assimetry_DTPV_T0,0.039463
52,BEOA_T0,0.036982
28,SwingSpeed_D_DTPV_T0,0.035598
71,SF36_GHP_T0,0.032914
14,PeakAngVelocity_D_TS_T0,0.031757
29,MinToe_D_DTPV_T0,0.031065


In [119]:
generate_experiment(T0_df, classifier=RandomForestClassifier(n_estimators=50))

Accuracy: 0.43 (+/- 0.26)
F1 Micro: 0.44 (+/- 0.11)
Precision: 0.42 (+/- 0.19)
Recall: 0.49 (+/- 0.43)
ROC: 0.47 (+/- 0.31)


,Features,Score
56,BIOF_T0,0.099112
58,CTSIB_Total_T0,0.081361
15,SwingSpeed_D_TS_T0,0.054231
64,SF36_PF_T0,0.042792
19,Gait_Assimetry_DTPV_T0,0.039463
52,BEOA_T0,0.036982
28,SwingSpeed_D_DTPV_T0,0.035598
71,SF36_GHP_T0,0.032914
14,PeakAngVelocity_D_TS_T0,0.031757
29,MinToe_D_DTPV_T0,0.031065


## Without Relief (feature selection)

In [120]:
generate_experiment(T0_df, classifier=SVC(kernel='poly'), relief=False)

Accuracy: 0.42 (+/- 0.16)
F1 Micro: 0.42 (+/- 0.16)
Precision: 0.36 (+/- 0.39)
Recall: 0.53 (+/- 0.66)
ROC: 0.34 (+/- 0.18)


In [121]:
generate_experiment(T0_df, classifier=SVC(kernel='sigmoid'), relief=False)

Accuracy: 0.48 (+/- 0.13)
F1 Micro: 0.48 (+/- 0.13)
Precision: 0.39 (+/- 0.40)
Recall: 0.60 (+/- 0.67)
ROC: 0.47 (+/- 0.26)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [122]:
generate_experiment(T0_df, classifier=SVC(kernel='linear'), relief=False)

Accuracy: 0.54 (+/- 0.14)
F1 Micro: 0.54 (+/- 0.14)
Precision: 0.55 (+/- 0.14)
Recall: 0.53 (+/- 0.22)
ROC: 0.53 (+/- 0.19)


In [124]:
generate_experiment(T0_df, classifier=RandomForestClassifier(n_estimators=100), relief=False)

Accuracy: 0.40 (+/- 0.14)
F1 Micro: 0.46 (+/- 0.23)
Precision: 0.44 (+/- 0.20)
Recall: 0.38 (+/- 0.32)
ROC: 0.35 (+/- 0.37)


In [125]:
generate_experiment(T0_df, classifier=RandomForestClassifier(n_estimators=50), relief=False)

Accuracy: 0.42 (+/- 0.18)
F1 Micro: 0.40 (+/- 0.21)
Precision: 0.37 (+/- 0.23)
Recall: 0.53 (+/- 0.34)
ROC: 0.36 (+/- 0.26)


# Experiment #3 
- Using all subjects that went to all measurements to predict Group

In [126]:
all_Ts = df.drop(labels=['Grupo', 'Sexo', 'Missing Values'], axis=1)
all_Ts = all_Ts.dropna()
print(len(all_Ts))

17


In [127]:
generate_experiment(all_Ts, classifier=SVC(kernel='poly'))

Accuracy: 0.63 (+/- 0.67)
F1 Micro: 0.63 (+/- 0.67)
Precision: 0.70 (+/- 0.80)
Recall: 0.60 (+/- 0.75)
ROC: 0.75 (+/- 0.77)


,Features,Score
342,SF36_Pain_T12,0.353758
97,StrideLength_D_DTPV_T9,0.308320
274,CTSIB_Total_T0,0.283497
303,GDS_T12,0.266340
337,SF36_VEF_T9,0.264706
255,BEDO_T3,0.264706
163,StrideLength_D_DTPF_T12,0.242599
335,SF36_GMH_T9,0.235294
225,EERE_T3,0.228758
315,SF36_Pain_T3,0.223856


In [128]:
generate_experiment(all_Ts, classifier=SVC(kernel='sigmoid'))

Accuracy: 0.40 (+/- 0.16)
F1 Micro: 0.40 (+/- 0.16)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.07 (+/- 0.27)
Recall: 0.20 (+/- 0.80)
ROC: 0.30 (+/- 0.80)


,Features,Score
342,SF36_Pain_T12,0.353758
97,StrideLength_D_DTPV_T9,0.308320
274,CTSIB_Total_T0,0.283497
303,GDS_T12,0.266340
337,SF36_VEF_T9,0.264706
255,BEDO_T3,0.264706
163,StrideLength_D_DTPF_T12,0.242599
335,SF36_GMH_T9,0.235294
225,EERE_T3,0.228758
315,SF36_Pain_T3,0.223856


In [129]:
generate_experiment(all_Ts, classifier=SVC(kernel='linear'))

Accuracy: 0.67 (+/- 0.37)
F1 Micro: 0.67 (+/- 0.37)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.50 (+/- 0.63)
Recall: 0.60 (+/- 0.75)
ROC: 0.75 (+/- 0.63)


,Features,Score
342,SF36_Pain_T12,0.353758
97,StrideLength_D_DTPV_T9,0.308320
274,CTSIB_Total_T0,0.283497
303,GDS_T12,0.266340
337,SF36_VEF_T9,0.264706
255,BEDO_T3,0.264706
163,StrideLength_D_DTPF_T12,0.242599
335,SF36_GMH_T9,0.235294
225,EERE_T3,0.228758
315,SF36_Pain_T3,0.223856


In [131]:
generate_experiment(all_Ts, classifier=RandomForestClassifier(n_estimators=100))

Accuracy: 0.42 (+/- 0.30)
F1 Micro: 0.48 (+/- 0.34)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.20 (+/- 0.80)
Recall: 0.10 (+/- 0.40)
ROC: 0.40 (+/- 0.75)


,Features,Score
342,SF36_Pain_T12,0.353758
97,StrideLength_D_DTPV_T9,0.308320
274,CTSIB_Total_T0,0.283497
303,GDS_T12,0.266340
337,SF36_VEF_T9,0.264706
255,BEDO_T3,0.264706
163,StrideLength_D_DTPF_T12,0.242599
335,SF36_GMH_T9,0.235294
225,EERE_T3,0.228758
315,SF36_Pain_T3,0.223856


In [132]:
generate_experiment(all_Ts, classifier=RandomForestClassifier(n_estimators=50))

Accuracy: 0.37 (+/- 0.31)
F1 Micro: 0.67 (+/- 0.56)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.20 (+/- 0.80)
Recall: 0.20 (+/- 0.49)
ROC: 0.50 (+/- 0.89)


,Features,Score
342,SF36_Pain_T12,0.353758
97,StrideLength_D_DTPV_T9,0.308320
274,CTSIB_Total_T0,0.283497
303,GDS_T12,0.266340
337,SF36_VEF_T9,0.264706
255,BEDO_T3,0.264706
163,StrideLength_D_DTPF_T12,0.242599
335,SF36_GMH_T9,0.235294
225,EERE_T3,0.228758
315,SF36_Pain_T3,0.223856


## Without Relief (feature selection)

In [133]:
generate_experiment(all_Ts, classifier=SVC(kernel='poly'), relief=False)

Accuracy: 0.33 (+/- 0.37)
F1 Micro: 0.33 (+/- 0.37)
Precision: 0.00 (+/- 0.00)
Recall: 0.00 (+/- 0.00)
ROC: 0.25 (+/- 0.45)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [134]:
generate_experiment(all_Ts, classifier=SVC(kernel='sigmoid'), relief=False)

Accuracy: 0.33 (+/- 0.37)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1 Micro: 0.33 (+/- 0.37)
Precision: 0.07 (+/- 0.27)
Recall: 0.20 (+/- 0.80)
ROC: 0.25 (+/- 0.45)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [135]:
generate_experiment(all_Ts, classifier=SVC(kernel='linear'), relief=False)

Accuracy: 0.27 (+/- 0.45)
F1 Micro: 0.27 (+/- 0.45)
Precision: 0.30 (+/- 0.49)
Recall: 0.30 (+/- 0.49)
ROC: 0.15 (+/- 0.40)


In [137]:
generate_experiment(all_Ts, classifier=RandomForestClassifier(n_estimators=100), relief=False)

Accuracy: 0.28 (+/- 0.33)
F1 Micro: 0.40 (+/- 0.45)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.10 (+/- 0.40)
Recall: 0.20 (+/- 0.80)
ROC: 0.35 (+/- 0.60)


In [138]:
generate_experiment(all_Ts, classifier=RandomForestClassifier(n_estimators=50), relief=False)

Accuracy: 0.35 (+/- 0.45)
F1 Micro: 0.42 (+/- 0.30)


/home/sarai/.virtualenvs/relief/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.40 (+/- 0.98)
Recall: 0.10 (+/- 0.40)
ROC: 0.38 (+/- 0.39)
